## Урок 3. 
### Парсинг данных. HTML, Beautiful Soap

#### Задание 1
**Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
Наименование вакансии.
Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
Ссылку на саму вакансию.
Сайт, откуда собрана вакансия.
По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.**

In [6]:
!pip install bs4

import requests
from pprint import pprint
from lxml import html
from bs4 import BeautifulSoup as bs
import requests
from time import sleep

Defaulting to user installation because normal site-packages is not writeable


In [7]:
url = 'https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=python&no_magic=true&L_save_area=true&items_on_page=20'
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 YaBrowser/23.1.1.1138 Yowser/2.5 Safari/537.36'
headers = {'User-Agent': user_agent, }
response = requests.get(url, headers=headers)

In [8]:
response

<Response [200]>

In [9]:
response.url

'https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=python&no_magic=true&L_save_area=true&items_on_page=20'

In [10]:
soup = bs(response.text, 'html.parser')

In [11]:
soup

<!DOCTYPE html>

<html class="desktop" lang="ru"><!--request: 167674095700742927f988504308d0d3--><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><title data-reactroot="" data-rh="">Работа в Санкт-Петербурге, поиск персонала и публикация вакансий - spb.hh.ru</title><meta content="summary_large_image" name="twitter:card"/><meta content="spb.hh.ru — сервис, который помогает найти работу и подобрать персонал в Санкт-Петербурге более 20 лет! Создавайте резюме и откликайтесь на вакансии. Набирайте сотрудников и публикуйте вакансии." data-reactroot="" data-rh="" name="description"/><meta content="работа, вакансии, работа, поиск вакансий, резюме, работы, работу, работ, ищу работу, поиск" data-reactroot="" data-rh="" name="keywords"/><meta content="https://i.hh.ru/logos/openGraph/hh.ru.png?v=12012023?v=3" property="og:image"/><meta content="600" property="og:image:width"/><meta content="315" property="og:image:heigh

In [12]:
soup.head.parent.name

'html'

In [13]:
soup.parent == None

True

In [31]:
tag_h1 = soup.find('h1', {'data-qa':'bloko-header-3'})
print(tag_h1)
print(type(tag_h1))
print(f'Количество искомых элементов: {len(tag_h1)}')

<h1 class="bloko-header-section-3" data-qa="bloko-header-3">177 <span>вакансий</span> «python»</h1>
<class 'bs4.element.Tag'>
Количество искомых элементов: 3


In [15]:
index_page=1
print('Cтраница №%a, \nВакансия: %s , \nСсылка: %s'%(index_page, tag_h1, response.url))

Cтраница №1, 
Вакансия: <h1 class="bloko-header-section-3" data-qa="bloko-header-3">177 <span>вакансий</span> «python»</h1> , 
Ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=python&no_magic=true&L_save_area=true&items_on_page=20


In [16]:
link_page =  soup.find('a', {'data-qa':'pager-next'})
link_page

<a class="bloko-button" data-qa="pager-next" href="/search/vacancy?area=88&amp;search_field=name&amp;search_field=company_name&amp;search_field=description&amp;text=python&amp;no_magic=true&amp;L_save_area=true&amp;items_on_page=20&amp;page=1&amp;hhtmFrom=vacancy_search_list" rel="nofollow"><span>дальше</span></a>

In [17]:
vacancies = soup.find_all('div', {'class': 'vacancy-serp-item__layout'})
vacancies

[<div class="vacancy-serp-item__layout"><div class="vacancy-serp-item-body"><div class="vacancy-serp-item-body__main-info"><div class=""><h3 class="bloko-header-section-3" data-qa="bloko-header-3"><span data-page-analytics-event="vacancy_search_suitable_item"><a class="serp-item__title" data-qa="serp-item__title" href="https://spb.hh.ru/vacancy/77083921?from=vacancy_search_list&amp;query=python" target="_blank">Преподаватель Python</a></span></h3><div class="bloko-v-spacing bloko-v-spacing_base-1"></div><span class="bloko-header-section-3" data-qa="vacancy-serp__vacancy-compensation">от <!-- -->125 000<!-- --> <!-- -->руб.</span><div class="bloko-v-spacing bloko-v-spacing_base-3"></div></div><div class="vacancy-serp-item-company"><div class="vacancy-serp-item__info"><div class="bloko-v-spacing-container bloko-v-spacing-container_base-2"><div class="bloko-text"><div class="vacancy-serp-item__meta-info-company"><a class="bloko-link bloko-link_kind-tertiary" data-qa="vacancy-serp__vacancy

In [24]:
def parse_hh(url_page, headers, result=[], index_page=1):
    response = requests.get(url_page, headers=headers)
    if response.status_code != 200:
      print('Парсинг завершен')
      return result
    else:
      print('Cтраница №%d, ссылка: %s'%(index_page, response.url))

    soup = bs(response.content, 'html.parser')
    vacancies = soup.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for vacancy in vacancies:
      result.append(parse_vacancy_hh(vacancy))

    link_next_page = soup.find('a', {'data-qa': 'pager-next'})
    if link_next_page:
      link_next_page =  'https://spb.hh.ru' + link_next_page['href']
    else:
      print('Парсинг завершен')
      return result

    result = parse_hh(link_next_page, headers, result, index_page+1)   
    return result
     
def parse_vacancy_hh(soup_vacancy):
  sleep(0.1)
  vacancy_name = soup_vacancy.find('a').text

  vacancy_salary = soup_vacancy.find('span', {'class', 'bloko-header-section-3'})
  if vacancy_salary:
    vacancy_salary = vacancy_salary.text
    min_salary, max_salary, currency_salary = clean_salary(vacancy_salary)
  else:
    min_salary, max_salary, currency_salary = None, None, None

  vacancy_link = soup_vacancy.find('a')['href']
  
  return {
      'vacancy_name': vacancy_name,
      'vacancy_salary': vacancy_salary,
      'min_salary': min_salary,
      'max_salary': max_salary,
      'currency_salary': currency_salary,
      'vacancy_link': vacancy_link,
      'vacancy_source': 'hh.ru',
  }

def clean_salary(vacancy_salary_text, min_salary=None, max_salary=None, currency_salary=None):
  list_salary = vacancy_salary_text.replace('\u202f', '').split()
  for i in range(len(list_salary) - 1):
    if list_salary[i] == 'от':
      min_salary = int(list_salary[i + 1])
    elif list_salary[i] == 'до':
      max_salary = int(list_salary[i + 1])
    elif list_salary[i] == '–':
      min_salary = int(list_salary[i - 1])
      max_salary = int(list_salary[i + 1])
  currency_salary = list_salary[-1]

  return min_salary, max_salary, currency_salary

In [25]:
result = parse_hh(url, headers)

Cтраница №1, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=python&no_magic=true&L_save_area=true&items_on_page=20
Cтраница №2, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=python&no_magic=true&L_save_area=true&items_on_page=20&page=1&hhtmFrom=vacancy_search_list
Cтраница №3, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=python&no_magic=true&L_save_area=true&items_on_page=20&page=2&hhtmFrom=vacancy_search_list
Cтраница №4, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=python&no_magic=true&L_save_area=true&items_on_page=20&page=3&hhtmFrom=vacancy_search_list
Cтраница №5, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=descripti

In [22]:
len(result)

174

In [26]:
result

[{'vacancy_name': 'Преподаватель Python',
  'vacancy_salary': 'от 125\u202f000 руб.',
  'min_salary': 125000,
  'max_salary': None,
  'currency_salary': 'руб.',
  'vacancy_link': 'https://spb.hh.ru/vacancy/77083921?from=vacancy_search_list&query=python',
  'vacancy_source': 'hh.ru'},
 {'vacancy_name': 'Team Lead Python',
  'vacancy_salary': 'от 400\u202f000 руб.',
  'min_salary': 400000,
  'max_salary': None,
  'currency_salary': 'руб.',
  'vacancy_link': 'https://spb.hh.ru/vacancy/76930290?from=vacancy_search_list&query=python',
  'vacancy_source': 'hh.ru'},
 {'vacancy_name': 'Python разработчик',
  'vacancy_salary': None,
  'min_salary': None,
  'max_salary': None,
  'currency_salary': None,
  'vacancy_link': 'https://spb.hh.ru/vacancy/75951525?from=vacancy_search_list&query=python',
  'vacancy_source': 'hh.ru'},
 {'vacancy_name': 'Middle Python developer (Казань)',
  'vacancy_salary': 'от 160\u202f000 руб.',
  'min_salary': 160000,
  'max_salary': None,
  'currency_salary': 'руб.',
